In [1]:
from datetime import datetime
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
import pandas as pd
import yahoo_fin.stock_info as si
from utils.utils_bigquery import *
from datetime import *

In [2]:
key_path = key_path
project = project_id

# Sources
indicators_table = 'silver_currency_data'
indicators_dataset = 'silver'

# Tables id
table_conca_indicators = f'{project}.{indicators_dataset}.{indicators_table}'

# Loading tables
table_to_save = 'silver_itm_currency_data'
dataset_to_save = 'silver'

In [3]:
# Conectamos con Bigquery
bigquery = BigQueryUtils(key_path)

In [4]:
df = bigquery.run_query(
    f"""
    SELECT
        *
    FROM {table_conca_indicators}  
    """
)

df

,Date,USDCHF,USDCNY,USDEUR,USDGBP,USDJPY
0,2020-02-28 00:00:00+00:00,0.96875,7.0035,0.90932,0.77586,109.660004
1,2017-03-16 00:00:00+00:00,1.00000,6.9136,0.93128,0.81405,113.402000
2,2019-02-21 00:00:00+00:00,1.00000,6.7203,0.88134,0.76657,110.764999
3,2024-06-04 00:00:00+00:00,0.89502,7.2420,0.91681,0.78063,156.076004
4,2020-06-09 00:00:00+00:00,0.95752,7.0703,0.88452,0.78522,108.408997
...,...,...,...,...,...,...
2515,2020-11-06 00:00:00+00:00,0.90417,6.6052,0.84620,0.76107,103.602997
2516,2021-02-02 00:00:00+00:00,0.89648,6.4675,0.82859,0.73106,104.880997
2517,2022-11-04 00:00:00+00:00,1.01342,7.3008,1.02568,0.89516,148.393005
2518,2018-12-11 00:00:00+00:00,0.98950,6.9110,0.88055,0.79610,113.181000


In [5]:
df_unpivot = df.melt(id_vars=['Date'], var_name='currency', value_name='currency_value')

df_unpivot = df_unpivot.sort_values(by='Date')

df_unpivot

,Date,currency,currency_value
4666,2015-01-01 00:00:00+00:00,USDCNY,6.196100
9706,2015-01-01 00:00:00+00:00,USDGBP,0.641810
2146,2015-01-01 00:00:00+00:00,USDCHF,0.994110
7186,2015-01-01 00:00:00+00:00,USDEUR,0.826540
12226,2015-01-01 00:00:00+00:00,USDJPY,119.672997
...,...,...,...
9558,2024-09-02 00:00:00+00:00,USDGBP,0.761700
4518,2024-09-02 00:00:00+00:00,USDCNY,7.092100
12078,2024-09-02 00:00:00+00:00,USDJPY,146.380997
7038,2024-09-02 00:00:00+00:00,USDEUR,0.905400


In [6]:
# Guardamos los datos en bigquery
bigquery.save_dataframe(df_unpivot, project, dataset_to_save, table_to_save, if_exists='replace', schema=None)

100%|██████████| 1/1 [00:00<?, ?it/s]
